In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import anndata

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as tf
import timm
from torch.utils.data import Dataset, DataLoader
import tqdm

/home/matthieu.bernard/miniconda3/envs/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Dataset de test
X_test = torch.linspace(-10, 10, 100).view(-1, 1)
Y_test = X_test * 2

In [5]:
#Dataset
class MarkersDataset(Dataset):
    def __init__(self):
        self.X = torch.tensor(X_test)
        self.y = torch.tensor(Y_test)
        #self.X = torch.tensor(markers_df.to_numpy())
        #self.y = torch.tensor(clusters_df.to_numpy())

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, nb_if_markers, nb_imc_markers, hidden_dim=1000):
        super().__init__()

        self.if_to_hidden1 = nn.Linear(nb_if_markers,hidden_dim)
        self.hidden1_to_hidden2 = nn.Linear(hidden_dim,hidden_dim)
        self.hidden2_to_imc = nn.Linear(hidden_dim,nb_imc_markers)

        self.relu = nn.ReLU()

        self.net = nn.Sequential(self.if_to_hidden1,self.relu,
                                 self.hidden1_to_hidden2,self.relu,
                                 self.hidden2_to_imc)

    def forward(self,if_markers):
        predicted_imc = self.net(if_markers)
        
        return predicted_imc

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#number of markers 
nb_IF_markers = 1
nb_IMC_markers = 1

#Training parameters
n_epochs = 500
batch_size = 16
lr = 0.01 # Karpathy constant 3e-4

#dataset loading
dataset = MarkersDataset()
train_loader = DataLoader(dataset=dataset, batch_size = batch_size, shuffle=True)
model = SimpleNN(nb_IF_markers,nb_IMC_markers).to(device)
optimizer = optim.Adam(model.parameters(), lr = lr)
loss_f = nn.MSELoss(reduction = "sum")

/tmp/ipykernel_29891/3707492734.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X_test)
/tmp/ipykernel_29891/3707492734.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(Y_test)


In [16]:
loop = tqdm.tqdm(range(n_epochs))
for epoch in loop:
    model.train()
    predicted_imc = model(X_test)

    reconstruction_loss = loss_f(predicted_imc,Y_test)
    loss = reconstruction_loss # not VAE so no kl_div
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loop.set_postfix(loss=loss.item())

100%|██████████| 500/500 [00:31<00:00, 16.04it/s, loss=0.00284]


In [ ]:
directory = './nn_checkpoints/'

if not os.path.exists(directory):
        os.makedirs(directory)

nn_basename = 'nn_' + str(n_epochs) + 'epochs.pth'
nn_path = directory + nn_basename

torch.save({'tot_epoch':n_epochs,
            'model_state_dict' : model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss' : loss,
            },nn_path)

In [ ]:
nn_basename = 'nn_' + str(n_epochs) + 'epochs.pth'
nn_path = directory + nn_basename

checkpoint = torch.load(nn_path)